In [83]:
import yaml

# Monster没有继承，需要使用标签!!python/object:__main__.Monster
# 反序列化时，才能判断要序列化成的数据类型
class Monster:
    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks
    
    def __repr__(self):
        return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
            self.__class__.__name__, self.name, self.hp,
            self.ac, self.attacks
        )


# 对于load和dump方法，完全不需要提前知道类型的信息，只要它们继承了YAMLObject
# 就能被序列化和反序列化
monster_object_load_1 = yaml.load("""
!!python/object:__main__.Monster
    name: Cave spider1
    hp: #2d6
      - 2
      - 6
    ac: 16
    attacks:
      - BITE
      - HURT
    """, Loader=yaml.Loader)

monster_object_load_2 = yaml.load("""
!!python/object:__main__.Monster
    name: Cave spider2
    hp: [8,6] #2d6
    ac: 16
    attacks: [BITE,HURT]
    """, Loader=yaml.Loader)

print("---load yaml 序列化成对象---")
print(monster_object_load_1)
print(monster_object_load_2)

print("---dump 对象反序列化成yaml格式---")
monster_object = Monster('Cave lizard1', [3,6], 16, ['BITE','HURT'])
monster_object_dump_1 = yaml.dump(monster_object)
monster_object_dump_2 = yaml.dump(Monster(name='Cave lizard2', hp=[6,6], ac=16, attacks=['BITE','HURT']))
print(monster_object_dump_1)
print(monster_object_dump_2)

---load yaml 序列化成对象---
Monster(name='Cave spider1', hp=[2, 6], ac=16, attacks=['BITE', 'HURT'])
Monster(name='Cave spider2', hp=[8, 6], ac=16, attacks=['BITE', 'HURT'])
---dump 对象反序列化成yaml格式---
!!python/object:__main__.Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard1

!!python/object:__main__.Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 6
- 6
name: Cave lizard2



In [82]:
import yaml

# YAMLObject的任意子类型支持序列化与反序列化
# 继承了YAMLObject, 可以通过yaml_tag设置自定义标签
# 当然，不设置自定义标签，还是需要使用!!python/object:__main__.Monster2
class Monster2(yaml.YAMLObject):
    # 自定义定义标签
    yaml_tag = u'!Monster2'
    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks
    
    def __repr__(self):
        return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
            self.__class__.__name__, self.name, self.hp,
            self.ac, self.attacks
        )


# 对于load和dump方法，完全不需要提前知道类型的信息，只要它们继承了YAMLObject
# 就能被序列化和反序列化
monster2_object_load_1 = yaml.load("""
    !Monster2
    name: Cave spider1
    hp: #2d6
      - 2
      - 6
    ac: 16
    attacks:
      - BITE
      - HURT
    """, Loader=yaml.Loader)

monster2_object_load_2 = yaml.load("""
    !Monster2
    name: Cave spider2
    hp: [8,6] #2d6
    ac: 16
    attacks: [BITE,HURT]
    """, Loader=yaml.Loader)

print("---load yaml 序列化成对象---")
print(monster2_object_load_1)
print(monster2_object_load_2)

print("---dump 对象反序列化成yaml格式---")
monster2_object = Monster2('Cave lizard1', [3,6], 16, ['BITE','HURT'])
monster2_object_dump_1 = yaml.dump(monster2_object)
monster2_object_dump_2 = yaml.dump(Monster2(name='Cave lizard2', hp=[6,6], ac=16, attacks=['BITE','HURT']))
print(monster2_object_dump_1)
print(monster2_object_dump_2)



---load yaml 序列化成对象---
Monster2(name='Cave spider1', hp=[2, 6], ac=16, attacks=['BITE', 'HURT'])
Monster2(name='Cave spider2', hp=[8, 6], ac=16, attacks=['BITE', 'HURT'])
---dump 对象反序列化成yaml格式---
!Monster2
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard1

!Monster2
ac: 16
attacks:
- BITE
- HURT
hp:
- 6
- 6
name: Cave lizard2



In [89]:
# 所有用自定义的类的类型都是type
class MyClass:
    pass

my_class_instance = MyClass()
print(type(my_class_instance))
print(type(MyClass))

class MyClass1:
    data = 1

my_class1_instance = MyClass1()
print("MyClass1 is : ", MyClass1)
print("my_class1_instance is : ", my_class1_instance)
print("my_class1_instance.data is : ", my_class1_instance.data)

# 用户自定义的类，是type类的__call__运算符重载，
# 如下
MyClass2 = type("MyClass1",(),{"data": 2})
my_class2_instance = MyClass2()
print("MyClass2 is : ", MyClass2)
print("my_class2_instance is : ", my_class2_instance)
print("my_class2_instance.data is : ", my_class2_instance.data)

# metaclass是type的子类，通过替换type的__call__运算符重载机制，
# 从而构造想要构造的python类

<class '__main__.MyClass'>
<class 'type'>
MyClass1 is :  <class '__main__.MyClass1'>
my_class1_instance is :  <__main__.MyClass1 object at 0x10f5733d0>
my_class1_instance.data is :  1
MyClass2 is :  <class '__main__.MyClass1'>
my_class2_instance is :  <__main__.MyClass1 object at 0x10f3d33d0>
my_class2_instance.data is :  2


#### `__init__()`的作用是初始化某个类的一个实例
#### `__call__()`的作用是使实例能够像函数一样被调用，同时不影响实例本身的生命周期（`__call__()`不影响一个实例的构造和析构）。但是`__call__()`可以用来改变实例的内部成员的值。


In [ ]:
# 可以在一个类中加入__metaclass__属性，python会使使用__metaclass__来创造类
# 当写下如下代码时，内存中还没有Foo这个类对象
# python会在Foo类定义中查找__metaclass__，存在则使用__metaclass__创造Foo类对象
# 不存在，则使用type来创造类对象
class Foo(object):
    __metaclass__= "something"

# 对于如下情况
# Foo1类定义中没有__metaclass__，到父类中查找__metaclass__
# 如果遍历父类都找不到__metaclass__，会在模块中寻找__metaclass__
# 如果还是找不到，会用内置的type来创建这个类对象
class Bar:
    pass

class Foo1(Bar):
    pass

# 元类可以修改类的行为，比较危险，慎用
# 参考 
# https://segmentfault.com/a/1190000007255412
# https://wiki.jikexueyuan.com/project/explore-python/Class/metaclass.html


